In [2]:
import pandas as pd

df = pd.read_excel(r'C:\Users\Abdykarim.D\PycharmProjects\robot-1157-DWH\working_path\Splitted\Алматинский филиал №1_2023-05-31.xlsx')
df

,Артикул,Подгруппа,Наименование товара,Ценовой сегмент,Филиал,Код товара,Код филиала,Учётные остатки,Фактические остатки,Свободные остатки,Кол-во проблемных
0,390510-210752,КАПУСТА,КАПУСТА БРОККОЛИ КАЗАХСТАН КГ,средний +,Алматинский филиал №1,210752,4,0,0.0,0,6062
1,390510-248017,КАПУСТА,КАПУСТА БРОККОЛИ КИТАЙ КГ,средний +,Алматинский филиал №1,248017,4,37994,37994.0,37994,31573
2,390510-210977,КАПУСТА,КАПУСТА КАЗАХСТАН КГ,эконом,Алматинский филиал №1,210977,91,166800,166800.0,166800,166800
3,390510-210977,КАПУСТА,КАПУСТА КАЗАХСТАН КГ,эконом,Алматинский филиал №1,210977,4,2600435,2600435.0,2595139,1890649
4,390510-210755,КАПУСТА,КАПУСТА КРАСНАЯ КАЗАХСТАН КГ,эконом,Алматинский филиал №1,210755,4,52470,52470.0,52470,14284
...,...,...,...,...,...,...,...,...,...,...,...
142,390502-74630,"ЛУК, ЧЕСНОК",ЧЕСНОК КАЗАХСТАН КГ,средний,Алматинский филиал №1,74630,4,393,393.0,393,8901
143,390502-74627,"ЛУК, ЧЕСНОК",ЧЕСНОК КИТАЙ КГ,средний,Алматинский филиал №1,74627,4,31251,31251.0,30847,14553
144,390502-74631,"ЛУК, ЧЕСНОК",ЧЕСНОК МОЛОДОЙ КАЗАХСТАН КГ,средний,Алматинский филиал №1,74631,4,61870,61870.0,61745,29626
145,390502-337716,"ЛУК, ЧЕСНОК",ЧЕСНОК МОЛОДОЙ КГ,средний,Алматинский филиал №1,337716,4,0,0.0,0,8815


In [121]:
import psycopg2
import csv
import os
from openpyxl import load_workbook

conn = psycopg2.connect(dbname='adb', host='172.16.10.22', port='5432',
                            user='rpa_robot', password='Qaz123123+')

cur = conn.cursor()
date = '2023-05-31'
cur.execute(f"""select dpl.article as "Артикул", pg1.name_group_level5 as "Подгруппа", dpl.name_wares as "Наименование товара", dpl.extdesc28 as "Ценовой сегмент", ds.sale_obj_name as "Филиал", fwb.source_product_id as "Код товара", fwb.source_store_id as "Код филиала", sum(quantity_warehouse_acc) as "Учётные остатки", sum(quantity_warehouse) as "Фактические остатки", sum(quantity_free) as "Свободные остатки", sum(quantity_problem) as "Кол-во проблемных"
from dwh_data.fact_wares_bal fwb
left join dwh_data.dim_store ds on ds.source_store_id = fwb.source_store_id and current_date between ds.datestart and ds.dateend 
left join dwh_data.dim_prod_group pg1 on pg1.source_prod_id = fwb.source_product_id and  fwb.rep_date between pg1.datestart and pg1.dateend
left join dwh_data.dim_product_list dpl on dpl.code_wares = fwb.source_product_id and current_date between dpl.datestart and dpl.dateend 
where fwb.rep_date = '{str(date)}' and pg1.code_group_level5 in ('1398', '1399', '1417', '1437', '3951', '1745', '1746', '1747', '1754')
and ds.sale_obj_name = ds.sale_obj_name and dpl.extdesc28 = 'эконом'
group by fwb.source_product_id, pg1.name_group_level5, dpl.name_wares, dpl.extdesc28, dpl.article, ds.sale_obj_name, fwb.source_store_id
order by dpl.name_wares;
""")

rows = cur.fetchall()

with open(os.path.join(r'C:\Users\Abdykarim.D\PycharmProjects\robot-1157-DWH\working_path', 'all1.csv'), 'w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)

    for row in rows:
        writer.writerow(row)

cur.close()
conn.close()

In [122]:
df = pd.read_csv(r'C:\Users\Abdykarim.D\PycharmProjects\robot-1157-DWH\working_path\all1.csv', header=None)
df.columns = ["Артикул", "Подгруппа", "Наименование товара", "Ценовой сегмент", "Филиал", "Код товара", "Код филиала", "Учётные остатки", "Фактические остатки", "Свободные остатки", "Кол-во проблемных"]
df

,Артикул,Подгруппа,Наименование товара,Ценовой сегмент,Филиал,Код товара,Код филиала,Учётные остатки,Фактические остатки,Свободные остатки,Кол-во проблемных
0,260102-250834,КРУПА ГРЕЧНЕВАЯ ВЕС,ГРЕЧКА 25 КГ АЛТЫН ДАЛА,эконом,Алматинский филиал №2,250834,96,25000.0,24975.0,25000.0,0.0
1,390510-305609,КАПУСТА,КАПУСТА В ПЛЕНКЕ КАЗАХСТАН КГ,эконом,Астанинский филиал №4,305609,832,0.0,0.0,0.0,899835.0
2,390510-305609,КАПУСТА,КАПУСТА В ПЛЕНКЕ КАЗАХСТАН КГ,эконом,Астанинский филиал №28,305609,1568,1395.0,1395.0,1395.0,0.0
3,390510-305609,КАПУСТА,КАПУСТА В ПЛЕНКЕ КАЗАХСТАН КГ,эконом,Алматы ГО + РЦ,305609,95,0.0,0.0,0.0,3667871.0
4,390510-305609,КАПУСТА,КАПУСТА В ПЛЕНКЕ КАЗАХСТАН КГ,эконом,Алматинский филиал №7,305609,1703,0.0,0.0,0.0,5626.0
...,...,...,...,...,...,...,...,...,...,...,...
11734,3951-239372,СОЛЬ ОБЫЧНАЯ,СОЛЬ ХАН ТУЗ ПИЩЕВАЯ ЙОДИРОВАННАЯ 1КГ П/П,эконом,Астанинский филиал №6,239372,885,2.0,2.0,2.0,0.0
11735,3951-239372,СОЛЬ ОБЫЧНАЯ,СОЛЬ ХАН ТУЗ ПИЩЕВАЯ ЙОДИРОВАННАЯ 1КГ П/П,эконом,Алматинский филиал №39,239372,1069,30.0,30.0,30.0,0.0
11736,3951-239372,СОЛЬ ОБЫЧНАЯ,СОЛЬ ХАН ТУЗ ПИЩЕВАЯ ЙОДИРОВАННАЯ 1КГ П/П,эконом,Карагандинский филиал №4,239372,1208,0.0,1.0,0.0,1.0
11737,3951-239372,СОЛЬ ОБЫЧНАЯ,СОЛЬ ХАН ТУЗ ПИЩЕВАЯ ЙОДИРОВАННАЯ 1КГ П/П,эконом,Филиал №1 в г. Конаев,239372,2183,143.0,143.0,143.0,0.0


In [123]:
df[df['Наименование товара'].str.lower().str.contains('масло')]

,Артикул,Подгруппа,Наименование товара,Ценовой сегмент,Филиал,Код товара,Код филиала,Учётные остатки,Фактические остатки,Свободные остатки,Кол-во проблемных
2234,260501-217269,МАСЛО ПОДСОЛНЕЧНОЕ,МАСЛО ECOМАЙ ПОДСОЛНЕЧН РАФ 5Л П/Б,эконом,Алматинский филиал №19,217269,631,0.0,-2.0,0.0,0.0
2235,260501-319792,МАСЛО ПОДСОЛНЕЧНОЕ,МАСЛО MARKA ONE ПОДСОЛНЕЧНОЕ РАФ/ДЕЗ 0.9Л,эконом,Астанинский филиал №48,319792,2018,5.0,5.0,5.0,0.0
2236,260501-319792,МАСЛО ПОДСОЛНЕЧНОЕ,МАСЛО MARKA ONE ПОДСОЛНЕЧНОЕ РАФ/ДЕЗ 0.9Л,эконом,Астанинский филиал №6,319792,885,43.0,58.0,43.0,15.0
2237,260501-319792,МАСЛО ПОДСОЛНЕЧНОЕ,МАСЛО MARKA ONE ПОДСОЛНЕЧНОЕ РАФ/ДЕЗ 0.9Л,эконом,Астанинский филиал №44,319792,2027,15.0,15.0,15.0,0.0
2238,260501-319792,МАСЛО ПОДСОЛНЕЧНОЕ,МАСЛО MARKA ONE ПОДСОЛНЕЧНОЕ РАФ/ДЕЗ 0.9Л,эконом,Шымкентский филиал №15,319792,2127,45.0,48.0,45.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...
7546,260501-289059,МАСЛО ПОДСОЛНЕЧНОЕ,МАСЛО ЮЖНЫЙ ПОЛЮС ПОДСОЛНЕЧНОЕ РАФИНИРОВАННОЕ ...,эконом,Астанинский филиал №54,289059,2059,6.0,6.0,6.0,0.0
7547,260501-289059,МАСЛО ПОДСОЛНЕЧНОЕ,МАСЛО ЮЖНЫЙ ПОЛЮС ПОДСОЛНЕЧНОЕ РАФИНИРОВАННОЕ ...,эконом,Алматинский филиал №51,289059,1638,1.0,0.0,1.0,0.0
7548,260501-289059,МАСЛО ПОДСОЛНЕЧНОЕ,МАСЛО ЮЖНЫЙ ПОЛЮС ПОДСОЛНЕЧНОЕ РАФИНИРОВАННОЕ ...,эконом,Астанинский филиал №7,289059,889,1.0,0.0,1.0,0.0
7549,260501-289059,МАСЛО ПОДСОЛНЕЧНОЕ,МАСЛО ЮЖНЫЙ ПОЛЮС ПОДСОЛНЕЧНОЕ РАФИНИРОВАННОЕ ...,эконом,Астанинский филиал №35,289059,1944,15.0,15.0,15.0,0.0


In [32]:
df.columns

Index(['Артикул', 'Подгруппа', 'Наименование товара', 'Ценовой сегмент',
       'Филиал', 'Код товара', 'Код филиала', 'Учётные остатки',
       'Фактические остатки', 'Свободные остатки', 'Кол-во проблемных'],
      dtype='object')

In [136]:
df = pd.read_csv(r'C:\Users\Abdykarim.D\PycharmProjects\robot-1157-DWH\working_path\all.csv', header=None)
df.columns = ["Артикул", "Подгруппа", "Наименование товара", "Ценовой сегмент", "Филиал", "Код товара", "Код филиала", "Учётные остатки", "Фактические остатки", "Свободные остатки", "Кол-во проблемных"]
df

,Артикул,Подгруппа,Наименование товара,Ценовой сегмент,Филиал,Код товара,Код филиала,Учётные остатки,Фактические остатки,Свободные остатки,Кол-во проблемных
0,260102-250834,КРУПА ГРЕЧНЕВАЯ ВЕС,ГРЕЧКА 25 КГ АЛТЫН ДАЛА,эконом,Алматинский филиал №2,250834,96,25000.0,24975.0,25000.0,0.0
1,390510-305609,КАПУСТА,КАПУСТА В ПЛЕНКЕ КАЗАХСТАН КГ,эконом,Астанинский филиал №4,305609,832,0.0,0.0,0.0,899835.0
2,390510-305609,КАПУСТА,КАПУСТА В ПЛЕНКЕ КАЗАХСТАН КГ,эконом,Астанинский филиал №28,305609,1568,1395.0,1395.0,1395.0,0.0
3,390510-305609,КАПУСТА,КАПУСТА В ПЛЕНКЕ КАЗАХСТАН КГ,эконом,Алматы ГО + РЦ,305609,95,0.0,0.0,0.0,3667871.0
4,390510-305609,КАПУСТА,КАПУСТА В ПЛЕНКЕ КАЗАХСТАН КГ,эконом,Алматинский филиал №7,305609,1703,0.0,0.0,0.0,5626.0
...,...,...,...,...,...,...,...,...,...,...,...
11734,3951-239372,СОЛЬ ОБЫЧНАЯ,СОЛЬ ХАН ТУЗ ПИЩЕВАЯ ЙОДИРОВАННАЯ 1КГ П/П,эконом,Астанинский филиал №6,239372,885,2.0,2.0,2.0,0.0
11735,3951-239372,СОЛЬ ОБЫЧНАЯ,СОЛЬ ХАН ТУЗ ПИЩЕВАЯ ЙОДИРОВАННАЯ 1КГ П/П,эконом,Алматинский филиал №39,239372,1069,30.0,30.0,30.0,0.0
11736,3951-239372,СОЛЬ ОБЫЧНАЯ,СОЛЬ ХАН ТУЗ ПИЩЕВАЯ ЙОДИРОВАННАЯ 1КГ П/П,эконом,Карагандинский филиал №4,239372,1208,0.0,1.0,0.0,1.0
11737,3951-239372,СОЛЬ ОБЫЧНАЯ,СОЛЬ ХАН ТУЗ ПИЩЕВАЯ ЙОДИРОВАННАЯ 1КГ П/П,эконом,Филиал №1 в г. Конаев,239372,2183,143.0,143.0,143.0,0.0


In [137]:
df[df['Наименование товара'].str.lower().str.contains('масло')]

,Артикул,Подгруппа,Наименование товара,Ценовой сегмент,Филиал,Код товара,Код филиала,Учётные остатки,Фактические остатки,Свободные остатки,Кол-во проблемных
2234,260501-217269,МАСЛО ПОДСОЛНЕЧНОЕ,МАСЛО ECOМАЙ ПОДСОЛНЕЧН РАФ 5Л П/Б,эконом,Алматинский филиал №19,217269,631,0.0,-2.0,0.0,0.0
2235,260501-319792,МАСЛО ПОДСОЛНЕЧНОЕ,МАСЛО MARKA ONE ПОДСОЛНЕЧНОЕ РАФ/ДЕЗ 0.9Л,эконом,Астанинский филиал №48,319792,2018,5.0,5.0,5.0,0.0
2236,260501-319792,МАСЛО ПОДСОЛНЕЧНОЕ,МАСЛО MARKA ONE ПОДСОЛНЕЧНОЕ РАФ/ДЕЗ 0.9Л,эконом,Астанинский филиал №6,319792,885,43.0,58.0,43.0,15.0
2237,260501-319792,МАСЛО ПОДСОЛНЕЧНОЕ,МАСЛО MARKA ONE ПОДСОЛНЕЧНОЕ РАФ/ДЕЗ 0.9Л,эконом,Астанинский филиал №44,319792,2027,15.0,15.0,15.0,0.0
2238,260501-319792,МАСЛО ПОДСОЛНЕЧНОЕ,МАСЛО MARKA ONE ПОДСОЛНЕЧНОЕ РАФ/ДЕЗ 0.9Л,эконом,Шымкентский филиал №15,319792,2127,45.0,48.0,45.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...
7546,260501-289059,МАСЛО ПОДСОЛНЕЧНОЕ,МАСЛО ЮЖНЫЙ ПОЛЮС ПОДСОЛНЕЧНОЕ РАФИНИРОВАННОЕ ...,эконом,Астанинский филиал №54,289059,2059,6.0,6.0,6.0,0.0
7547,260501-289059,МАСЛО ПОДСОЛНЕЧНОЕ,МАСЛО ЮЖНЫЙ ПОЛЮС ПОДСОЛНЕЧНОЕ РАФИНИРОВАННОЕ ...,эконом,Алматинский филиал №51,289059,1638,1.0,0.0,1.0,0.0
7548,260501-289059,МАСЛО ПОДСОЛНЕЧНОЕ,МАСЛО ЮЖНЫЙ ПОЛЮС ПОДСОЛНЕЧНОЕ РАФИНИРОВАННОЕ ...,эконом,Астанинский филиал №7,289059,889,1.0,0.0,1.0,0.0
7549,260501-289059,МАСЛО ПОДСОЛНЕЧНОЕ,МАСЛО ЮЖНЫЙ ПОЛЮС ПОДСОЛНЕЧНОЕ РАФИНИРОВАННОЕ ...,эконом,Астанинский филиал №35,289059,1944,15.0,15.0,15.0,0.0


In [138]:
import re
df1 = df[df['Наименование товара'].str.lower().str.contains('масло')].copy()
kekus = []
for i in range(len(df1)):
    # print(df1.iloc[i])
    df1['Наименование товара'] = df1['Наименование товара'].str.replace(',', '.')
    
    multiplier = [float(s) for s in re.findall(r'[-+]?\d*\.\d+|\d+', df1['Наименование товара'].iloc[i])]
    
    if '0мл ' in df1['Наименование товара'].iloc[i].lower():
        df1['Фактические остатки'].iloc[i] *= float(max(multiplier)) / 1000.0
    else:
        df1['Фактические остатки'].iloc[i] *= float(max(multiplier))

C:\Users\Abdykarim.D\AppData\Local\Temp\ipykernel_11124\2456913035.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['Фактические остатки'].iloc[i] *= float(max(multiplier))
C:\Users\Abdykarim.D\AppData\Local\Temp\ipykernel_11124\2456913035.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['Фактические остатки'].iloc[i] *= float(max(multiplier))
C:\Users\Abdykarim.D\AppData\Local\Temp\ipykernel_11124\2456913035.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/inde

In [139]:
# df1 = df1.reset_index()
# df1 = df1.drop(['level_0'], axis=1)
df1

,Артикул,Подгруппа,Наименование товара,Ценовой сегмент,Филиал,Код товара,Код филиала,Учётные остатки,Фактические остатки,Свободные остатки,Кол-во проблемных
2234,260501-217269,МАСЛО ПОДСОЛНЕЧНОЕ,МАСЛО ECOМАЙ ПОДСОЛНЕЧН РАФ 5Л П/Б,эконом,Алматинский филиал №19,217269,631,0.0,-10.0,0.0,0.0
2235,260501-319792,МАСЛО ПОДСОЛНЕЧНОЕ,МАСЛО MARKA ONE ПОДСОЛНЕЧНОЕ РАФ/ДЕЗ 0.9Л,эконом,Астанинский филиал №48,319792,2018,5.0,4.5,5.0,0.0
2236,260501-319792,МАСЛО ПОДСОЛНЕЧНОЕ,МАСЛО MARKA ONE ПОДСОЛНЕЧНОЕ РАФ/ДЕЗ 0.9Л,эконом,Астанинский филиал №6,319792,885,43.0,52.2,43.0,15.0
2237,260501-319792,МАСЛО ПОДСОЛНЕЧНОЕ,МАСЛО MARKA ONE ПОДСОЛНЕЧНОЕ РАФ/ДЕЗ 0.9Л,эконом,Астанинский филиал №44,319792,2027,15.0,13.5,15.0,0.0
2238,260501-319792,МАСЛО ПОДСОЛНЕЧНОЕ,МАСЛО MARKA ONE ПОДСОЛНЕЧНОЕ РАФ/ДЕЗ 0.9Л,эконом,Шымкентский филиал №15,319792,2127,45.0,43.2,45.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...
7546,260501-289059,МАСЛО ПОДСОЛНЕЧНОЕ,МАСЛО ЮЖНЫЙ ПОЛЮС ПОДСОЛНЕЧНОЕ РАФИНИРОВАННОЕ ...,эконом,Астанинский филиал №54,289059,2059,6.0,30.0,6.0,0.0
7547,260501-289059,МАСЛО ПОДСОЛНЕЧНОЕ,МАСЛО ЮЖНЫЙ ПОЛЮС ПОДСОЛНЕЧНОЕ РАФИНИРОВАННОЕ ...,эконом,Алматинский филиал №51,289059,1638,1.0,0.0,1.0,0.0
7548,260501-289059,МАСЛО ПОДСОЛНЕЧНОЕ,МАСЛО ЮЖНЫЙ ПОЛЮС ПОДСОЛНЕЧНОЕ РАФИНИРОВАННОЕ ...,эконом,Астанинский филиал №7,289059,889,1.0,0.0,1.0,0.0
7549,260501-289059,МАСЛО ПОДСОЛНЕЧНОЕ,МАСЛО ЮЖНЫЙ ПОЛЮС ПОДСОЛНЕЧНОЕ РАФИНИРОВАННОЕ ...,эконом,Астанинский филиал №35,289059,1944,15.0,75.0,15.0,0.0


In [134]:
for i in df1.index:
    df.loc[i, 'Фактические остатки'] = df1['Фактические остатки'].iloc[i - 2234]    

In [135]:
df[df['Наименование товара'].str.lower().str.contains('масло')]

,Артикул,Подгруппа,Наименование товара,Ценовой сегмент,Филиал,Код товара,Код филиала,Учётные остатки,Фактические остатки,Свободные остатки,Кол-во проблемных
2234,260501-217269,МАСЛО ПОДСОЛНЕЧНОЕ,МАСЛО ECOМАЙ ПОДСОЛНЕЧН РАФ 5Л П/Б,эконом,Алматинский филиал №19,217269,631,0.0,-10.0,0.0,0.0
2235,260501-319792,МАСЛО ПОДСОЛНЕЧНОЕ,МАСЛО MARKA ONE ПОДСОЛНЕЧНОЕ РАФ/ДЕЗ 0.9Л,эконом,Астанинский филиал №48,319792,2018,5.0,4.5,5.0,0.0
2236,260501-319792,МАСЛО ПОДСОЛНЕЧНОЕ,МАСЛО MARKA ONE ПОДСОЛНЕЧНОЕ РАФ/ДЕЗ 0.9Л,эконом,Астанинский филиал №6,319792,885,43.0,52.2,43.0,15.0
2237,260501-319792,МАСЛО ПОДСОЛНЕЧНОЕ,МАСЛО MARKA ONE ПОДСОЛНЕЧНОЕ РАФ/ДЕЗ 0.9Л,эконом,Астанинский филиал №44,319792,2027,15.0,13.5,15.0,0.0
2238,260501-319792,МАСЛО ПОДСОЛНЕЧНОЕ,МАСЛО MARKA ONE ПОДСОЛНЕЧНОЕ РАФ/ДЕЗ 0.9Л,эконом,Шымкентский филиал №15,319792,2127,45.0,43.2,45.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...
7546,260501-289059,МАСЛО ПОДСОЛНЕЧНОЕ,МАСЛО ЮЖНЫЙ ПОЛЮС ПОДСОЛНЕЧНОЕ РАФИНИРОВАННОЕ ...,эконом,Астанинский филиал №54,289059,2059,6.0,30.0,6.0,0.0
7547,260501-289059,МАСЛО ПОДСОЛНЕЧНОЕ,МАСЛО ЮЖНЫЙ ПОЛЮС ПОДСОЛНЕЧНОЕ РАФИНИРОВАННОЕ ...,эконом,Алматинский филиал №51,289059,1638,1.0,0.0,1.0,0.0
7548,260501-289059,МАСЛО ПОДСОЛНЕЧНОЕ,МАСЛО ЮЖНЫЙ ПОЛЮС ПОДСОЛНЕЧНОЕ РАФИНИРОВАННОЕ ...,эконом,Астанинский филиал №7,289059,889,1.0,0.0,1.0,0.0
7549,260501-289059,МАСЛО ПОДСОЛНЕЧНОЕ,МАСЛО ЮЖНЫЙ ПОЛЮС ПОДСОЛНЕЧНОЕ РАФИНИРОВАННОЕ ...,эконом,Астанинский филиал №35,289059,1944,15.0,75.0,15.0,0.0


5317